In [8]:
import pandas as pd
import numpy as np

Looking at the first record, here is a breakdown of the important values.

* ESS_Id, actionDescription, ecuSoftwareVersion, ecuSerialNumber, ecuModel, ecuMake, ecuSource, faultValue, and MCTNumber are unlikely to provide any predictive value.
* We can see the time of the event in the **EventTimeStamp** column. Note that this may be different from the **LocationTimeStamp** value, which indicates when the Latitude/Longitude values were recorded.
* The **spn** and **fmi** columns together indicate the type of fault, and there may be a description of that fault in the **eventDescription** column, although this column is sometimes missing.
* Faults are recorded when the light goes on and when it goes off, which is indicated by the **active** column, with True indicating the light turning on and False indicating turning off. The number of times the code has been set or unset is in the **faultValue** column, although this value can be unreliable. 
* Each truck has an identifier, the **EquipmentID** value.
* Each record can be linked to the on-board diagnostics data through the **RecordID** column.

In [3]:
faults = pd.read_csv("../data/J1939Faults.csv")

C:\Users\lexil\AppData\Local\Temp\ipykernel_73612\798942974.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv("../data/J1939Faults.csv")


In [4]:
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")
diagnostics = diagnostics.pivot(index='FaultId', columns='Name', values='Value')

In [5]:
faults_and_diagnostics = faults.merge(diagnostics, how='left', left_on='RecordID', right_on='FaultId')
faults_and_diagnostics['FullDerate'] = faults_and_diagnostics['spn'].apply(lambda x: 1 if x == 5246 else 0)
faults_and_diagnostics = faults_and_diagnostics.sort_values(['EquipmentID', 'LocationTimeStamp'])

In [10]:
faults_and_diagnostics['EventTimeStamp'] = pd.to_datetime(faults_and_diagnostics['EventTimeStamp'])

faults_and_diagnostics['NextDerateTime'] = faults_and_diagnostics.where(faults_and_diagnostics['FullDerate']==1)['EventTimeStamp']
faults_and_diagnostics['NextDerateTime'] = faults_and_diagnostics.groupby('EquipmentID')['NextDerateTime'].fillna(method='bfill')

faults_and_diagnostics['HoursUntilNextDerate'] = (faults_and_diagnostics['NextDerateTime'] - faults_and_diagnostics['EventTimeStamp']).dt.total_seconds()/3600.0
faults_and_diagnostics['DerateInNextTwoHours'] = np.where(faults_and_diagnostics['HoursUntilNextDerate'] <= 2, 1, 0)
faults_and_diagnostics.head()

C:\Users\lexil\AppData\Local\Temp\ipykernel_73612\4079547888.py:4: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  faults_and_diagnostics['NextDerateTime'] = faults_and_diagnostics.groupby('EquipmentID')['NextDerateTime'].fillna(method='bfill')
C:\Users\lexil\AppData\Local\Temp\ipykernel_73612\4079547888.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  faults_and_diagnostics['NextDerateTime'] = faults_and_diagnostics.groupby('EquipmentID')['NextDerateTime'].fillna(method='bfill')


,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,...,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,FullDerate,NextDerateTime,HoursUntilNextDerate,DerateInNextTwoHours
33355,34467,2092800,2015-04-26 06:16:16,High (Severity High) Engine Speed,NaN,unknown,unknown,unknown,unknown,49,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2016-07-11 10:53:16,10612.616667,0
35080,36192,2123154,2015-04-28 05:29:21,High (Severity High) Engine Speed,NaN,unknown,unknown,unknown,unknown,49,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2016-07-11 10:53:16,10565.398611,0
47186,48298,2346346,2015-05-10 07:11:34,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,11,...,False,NaN,4.728246,3276.75,8.8,0.87,0,2016-07-11 10:53:16,10275.695000,0
47213,48325,2346621,2015-05-10 07:59:25,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,11,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2016-07-11 10:53:16,10274.897500,0
48303,49415,2363162,2015-05-11 13:11:20,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,11,...,False,NaN,4.475814,3276.75,14.4,0.58,0,2016-07-11 10:53:16,10245.698889,0


In [12]:
faults_and_diagnostics.shape
print(faults_and_diagnostics.info())

print(faults_and_diagnostics.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 1187335 entries, 33355 to 4999
Data columns (total 48 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   RecordID                   1187335 non-null  int64         
 1   ESS_Id                     1187335 non-null  int64         
 2   EventTimeStamp             1187335 non-null  datetime64[ns]
 3   eventDescription           1126490 non-null  object        
 4   actionDescription          0 non-null        float64       
 5   ecuSoftwareVersion         891285 non-null   object        
 6   ecuSerialNumber            844318 non-null   object        
 7   ecuModel                   1122577 non-null  object        
 8   ecuMake                    1122577 non-null  object        
 9   ecuSource                  1187335 non-null  int64         
 10  spn                        1187335 non-null  int64         
 11  fmi                        1187335 non-nu

In [14]:
print("Full Derate Label Distribution:")
print(faults_and_diagnostics['FullDerate'].value_counts())

Full Derate Label Distribution:
FullDerate
0    1186140
1       1195
Name: count, dtype: int64
